In [1]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
#TODO How to get access to test set from CARLA ? 
name='credit'
dataset = pd.read_csv(f'../data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
continuous_wachter = dataset.drop(columns=['label']).columns
dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
dataset._df_test

,label,x1,x2,x3,x4,x6,x5,x7
3098,1.0,0.0,-15.412158,-0.251252,-2.373504,-1.320468,-6.657412,-3.923228
3395,0.0,1.0,-11.384438,-0.171782,-1.076637,1.857091,-0.587414,-3.302045
2611,1.0,0.0,19.592985,-0.169971,-1.727356,-1.622798,0.157537,-11.978340
3384,0.0,0.0,5.733882,-0.058570,2.818482,1.721801,-0.111029,0.216785
7794,0.0,1.0,-7.051590,0.141726,-0.157746,-0.535435,2.488507,-0.135469
...,...,...,...,...,...,...,...,...
2116,0.0,0.0,3.717445,0.116490,1.783268,1.086886,1.599088,-6.500056
15,0.0,0.0,14.890755,0.231051,2.750332,-1.325314,-0.691101,3.739968
2818,0.0,0.0,4.318079,-0.064028,0.257310,1.717130,0.693120,-3.454028
4241,0.0,0.0,-9.035398,-0.121817,0.931433,-0.736720,-1.992126,-8.912432


In [3]:
from string import digits
import re
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
auc=['0']
df=pd.DataFrame([])

for name in os.listdir('./Model/'):
    #print(name.split('.')[0][-1].isdigit())
    #print(name)
    if name.split('.')[0][-1].isdigit() or name.startswith('Linear')or name.startswith('Forest'):
        full_name=name
        model=torch.load(f'./Model/{name}')
        name=name.split('_')[1]
        name=name.replace('.pth','')
        #print('FIRST',name)
        #print(name[:-1])
        if name[-1].isdigit():
            name=name[:-1]
        #print('Second',name)

        dataset = pd.read_csv(f'../data/{name}/{name}.csv')
        continuous_wachter = dataset.drop(columns=['label']).columns
        scaler='Identity'
        if name == 'economic':
            scaler='MinMax'

        dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
        dataset=dataset._df_test
        data_x, data_y=dataset.drop(columns=['label']), dataset['label']
        data_pred= model.predict(data_x)
        model_name.append(full_name)
        print(data_pred.shape)
        print(data_y.shape)
        if len(data_pred.shape)>1:
            data_pred= np.argmax(data_pred, axis=1)

        accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
        precision.append(sklearn.metrics.precision_score(data_y,data_pred))
        recall.append(sklearn.metrics.recall_score(data_y,data_pred))
        auc.append(sklearn.metrics.roc_auc_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df['auc'] =auc
df

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model

(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice 

,model_name,accuracy,precision,recall,auc
0,0,0,0,0,0
1,Linear_credit.pth,0.7688,0.791246,0.740157,0.769266
2,Linear_economic.pth,0.8376,0.798025,0.830841,0.836749
3,Linear_sanity-3-lin.pth,0.996,0.997671,0.994582,0.996049
4,Linear_sanity-3-non-add.pth,0.9984,0.993818,1,0.998923
5,Linear_sanity-3-non-lin.pth,0.9968,1,0.993283,0.996641
6,MLP_credit0.pth,0.6824,0.692308,0.651125,0.682251
7,MLP_economic0.pth,0.8312,0.794118,0.823475,0.830285
8,MLP_sanity-3-lin0.pth,0.998,0.998444,0.997667,0.99801
9,MLP_sanity-3-non-add0.pth,0.9964,0.993528,0.991922,0.994898


In [4]:
# Table to Latex 
# Get header
st=''
col=[]
name=[]
df=df.iloc[1:]
n_saver=[]
key=[]
for index, row in df.iterrows():
    #print(row['model_name'].split('_'))
    #if st is '':
    #    st += row['model_name'].split('_')[1].replace('.pth','')[:-1]
    #else:
    #    st += ' & '+row['model_name'].split('_')[1].replace('.pth','')[:-1]
    if row['model_name'].split('_')[1].replace('.pth','')[-1].isdigit():
        print('is Digit')
        data=row['model_name'].split('_')[0]+ row['model_name'].split('_')[1].replace('.pth','')[-1]
    else: 
        data=row['model_name'].split('_')[0]
    col.append(data)
    name.append(row['model_name'].split('_')[1].replace('.pth','')[:-1])
    key.append(row['model_name'])
st+= ' \\\\ '
col= np.unique(col)
name = np.unique(name[name!='credi'])
name=['economic','credit','sanity-3-lin','sanity-3-non-lin','sanity-3-non-add']
for n in name :
    st += ' & '+ n

print('col',col)
print('name',name)
print('key', key)
for i in range(0,len(col)):
    st += col[i] 
    num=''
    if col[i][-1].isdigit():
        num = col[i][-1]
    for j in range(0,len(name)):
        #print(''col[i])
        #print(name[i])
        for k in key: 
            print(f'{name[j]}{num}')
            if col[i] in k and f'{name[j]}{num}' in k: 
                t=k

        print(t)
        #print(f'{col[i]}_{name[i]}')
        #print(str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0]))
        #print(df[df['model_name'] ==f'{col[i]}_{name[i]}']['accuracy'])
        st += ' & ' + str(df[df['model_name'] ==f'{t}']['accuracy'].values[0])
    st+=' \\\\ '
print(st)

is Digit
is Digit
is Digit
is Digit
is Digit
col ['Linear' 'MLP0']
name ['economic', 'credit', 'sanity-3-lin', 'sanity-3-non-lin', 'sanity-3-non-add']
key ['Linear_credit.pth', 'Linear_economic.pth', 'Linear_sanity-3-lin.pth', 'Linear_sanity-3-non-add.pth', 'Linear_sanity-3-non-lin.pth', 'MLP_credit0.pth', 'MLP_economic0.pth', 'MLP_sanity-3-lin0.pth', 'MLP_sanity-3-non-add0.pth', 'MLP_sanity-3-non-lin0.pth']
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
Linear_economic.pth
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
Linear_credit.pth
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
Linear_sanity-3-lin.pth
sanity-3-non-lin
sanity-3-non-lin
sanity-3-non-lin
sanity-3-non-lin
sanity-3-non-lin
sanity-3-non-lin
sanity-3-non-lin
sanity-3-non-lin
sanity-3-non-lin
sanity-3-non-lin
Linear_sanity-3-non-lin.pth
sanity-3-non-add
sanity-3-non-add
sani